<a href="https://colab.research.google.com/github/Ashar2shahid/blockchain-hub/blob/main/BigdataNotebook2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz

In [3]:
!tar xf spark-3.1.1-bin-hadoop2.7.tgz

In [4]:
!pip install -q findspark

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

In [6]:
import findspark
findspark.init()

In [7]:
findspark.find()

'/content/spark-3.1.1-bin-hadoop2.7'

In [8]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -y ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels

--2021-03-31 15:33:34--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.198.117.197, 174.129.26.222, 35.175.47.233, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.198.117.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14746350 (14M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  14.06M  44.0MB/s    in 0.3s    

2021-03-31 15:33:35 (44.0 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [14746350/14746350]

UnZip 6.00 of 20 April 2009, by Debian. Original by Info-ZIP.

Usage: unzip [-Z] [-opts[modifiers]] file[.zip] [list] [-x xlist] [-d exdir]
  Default action is to extract files in list, except those in xlist, to exdir;
  file[.zip] may be a wildcard.  -Z => ZipInfo mode ("unzip -Z" for usage).

  -p  extract files to pipe, no messages     -l  list files (short format)
  -f  freshen existing files, create none    -t  

In [9]:
ls

ngrok-stable-linux-amd64.zip  spark-3.1.1-bin-hadoop2.7/
sample_data/                  spark-3.1.1-bin-hadoop2.7.tgz


In [10]:
from pyspark import SparkContext,SparkConf
from pyspark.sql import SparkSession
import pyspark
conf = pyspark.SparkConf()
conf.set("spark.driver.memory","8G");
import numpy as np
sc = SparkContext.getOrCreate(conf=conf)
from pyspark.sql.functions import round, col

In [11]:
project_id = 'api3-307502'

In [12]:
from google.colab import auth
auth.authenticate_user()

In [13]:
!gcloud config set project {project_id}

Updated property [core/project].


In [14]:
!gsutil cp gs://ethereum-bigdata/token_transfer* /tmp/

Copying gs://ethereum-bigdata/token_transfer_2019_11_01__2020_01_01_000000000000...
Copying gs://ethereum-bigdata/token_transfer_2019_11_01__2020_01_01_000000000001...
Copying gs://ethereum-bigdata/token_transfer_2019_11_01__2020_01_01_000000000002...
Copying gs://ethereum-bigdata/token_transfer_2019_11_01__2020_01_01_000000000003...
- [4 files][  2.0 GiB/  2.0 GiB]   58.3 MiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://ethereum-bigdata/token_transfer_2019_11_01__2020_01_01_000000000004...
Copying gs://ethereum-bigdata/token_transfer_2019_11_01__2020_01_01_000000000005...
Copying gs://ethereum-bigdata/token_transfer_2019_11_01__2020_01_01_000000000006...
Copying gs://ethereum-bigdata/token_transfer_2019_11_01__2020_01_01_00000000000

In [15]:
spark = SparkSession(sc)

In [16]:
token_transfer = os.listdir('/tmp')

In [17]:
token_transfer = ["/tmp/"+x for x in token_transfer if "token_transfer_" in x]
token_transfer.sort()
token_transfer

['/tmp/token_transfer_2019_11_01__2020_01_01_000000000000',
 '/tmp/token_transfer_2019_11_01__2020_01_01_000000000001',
 '/tmp/token_transfer_2019_11_01__2020_01_01_000000000002',
 '/tmp/token_transfer_2019_11_01__2020_01_01_000000000003',
 '/tmp/token_transfer_2019_11_01__2020_01_01_000000000004',
 '/tmp/token_transfer_2019_11_01__2020_01_01_000000000005',
 '/tmp/token_transfer_2019_11_01__2020_01_01_000000000006',
 '/tmp/token_transfer_2019_11_01__2020_01_01_000000000007',
 '/tmp/token_transfer_2019_11_01__2020_01_01_000000000008',
 '/tmp/token_transfer_2019_11_01__2020_01_01_000000000009',
 '/tmp/token_transfer_2019_11_01__2020_01_01_000000000010',
 '/tmp/token_transfer_2019_11_01__2020_01_01_000000000011',
 '/tmp/token_transfer_2019_11_01__2020_01_01_000000000012',
 '/tmp/token_transfer_2019_11_01__2020_01_01_000000000013',
 '/tmp/token_transfer_2019_11_01__2020_01_01_000000000014',
 '/tmp/token_transfer_2019_11_01__2020_01_01_000000000015',
 '/tmp/token_transfer_2019_11_01__2020_0

In [18]:
from functools import reduce
dfToken_transfer = reduce(lambda x,y: x.unionAll(y), 
            [spark.read.format('com.databricks.spark.csv')
                       .load(f, header="true", inferSchema="true") 
             for f in token_transfer])

In [19]:
dfToken_transfer.createOrReplaceTempView("tokentransfer")
spark.sql("SELECT COUNT(*) FROM tokentransfer").show()

+---------+
| count(1)|
+---------+
|167801660|
+---------+



In [22]:
dfToken_transfer.printSchema()

root
 |-- token_address: string (nullable = true)
 |-- from_address: string (nullable = true)
 |-- to_address: string (nullable = true)
 |-- value: double (nullable = true)
 |-- transaction_hash: string (nullable = true)
 |-- log_index: integer (nullable = true)
 |-- block_timestamp: string (nullable = true)
 |-- block_number: integer (nullable = true)

